In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Setup

In [ ]:
import os
from glob import glob
import subprocess

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import cv2 as cv
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import resnet18
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
from torch.optim.lr_scheduler import _LRScheduler

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# User
IS_MAC = False

# File directories
ROOT_FOLDER = "Documents/CSC413_Project/" if not IS_MAC else "/content/drive/MyDrive/CSC413/Project/"  # Change this to where the project files are for you
DATASET = os.path.join(ROOT_FOLDER, "UTKFace/")

# Age Parameters
AGE_BIN_SIZE = 6
NUM_BINS = 20

# Forget Set Parameters
FORGET_SET_SIZE = 0.02
FORGET_SET_MALE_P = 0.5
FORGET_SET_RACE_P = [0.2, 0.2, 0.2, 0.2, 0.2]

# Model Parameters
BATCH_SIZE = 64

## Databases

In [ ]:
def get_files_in_folder(folder, use_glob=False):
  if use_glob:
    return sorted(glob(os.path.join(folder, "*")))[:30]
  return sorted(os.path.join(folder, f) for f in os.listdir(folder))

In [ ]:
from collections import defaultdict

data = get_files_in_folder(DATASET, use_glob=IS_MAC)

## CLEAN BAD DATA
bad_data = []

for path in data:
  path_copy = path
  image_name = path.rsplit("/")[-1]
  labels = image_name.split(".")[0].split("_")
  if len(labels) < 4:
    bad_data.append(path_copy)


if len(bad_data) > 0:
  print(f"YOU GOT SOME BAD PATHS SIR, SIZE: {len(bad_data)}")

for bad_path in bad_data:
  os.remove(bad_path)

data = get_files_in_folder(DATASET, use_glob=IS_MAC)

In [ ]:
def get_labels(path):
  """
  Returns a list of labels in the form of: [age, gender, race]
  """
  image_name = path.rsplit("/")[-1]
  labels = image_name.split(".")[0].split("_")

  if len(labels) < 4:
    # raise CustomError("This was a bad path")
    pass

  return [int(label) for label in labels[:3]]

In [ ]:
def train_test_split(data_dir_path, show_age_hist=False):
  image_paths = get_files_in_folder(data_dir_path, use_glob=IS_MAC)
  images = []
  image_labels = []
  i = 0
  females = 0

  for path in image_paths:
    images.append(path)
    age, gender, _ = get_labels(path)
    image_labels.append(age // AGE_BIN_SIZE)
    # TODO: think about how to get the model to also classify the race / gender
    # separate models? or all in one??
    i += 1
    females += gender

  if show_age_hist:
    plt.hist(image_labels, bins=max(image_labels) + 1, edgecolor='black')
    plt.xlabel('Bins')
    plt.ylabel('Frequency')
    plt.title('Histogram of Labels with 20 Bins')
    plt.show()
    print(len(image_paths), "female: ", females, "male: ", len(image_paths) - females)

  dataset = list(zip(images, image_labels))
  generator = torch.Generator().manual_seed(42)

  train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=generator)

  X_train, y_train = zip(*train_dataset)
  X_val, y_val = zip(*val_dataset)
  X_test, y_test = zip(*test_dataset)

  X_train, y_train = np.array(X_train), np.array(y_train)
  X_val, y_val = np.array(X_val), np.array(y_val)
  X_test, y_test = np.array(X_test), np.array(y_test)

  return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
def get_probs(X_train, male_prob, race_probs):
  sample_probs = []

  prob_tracker = {
      "gender": {0: male_prob, 1: 1 - male_prob},
      "race": {i: race_probs[i] for i in range(len(race_probs))}
  }

  gender_count = {
      0: 0,   # Male
      1: 0    # Female
  }
  race_count = {
      0: 0,   # White
      1: 0,   # Black
      2: 0,   # Asian
      3: 0,   # Indian
      4: 0    # Other
  }

  for path in X_train:
    age, gender, race = get_labels(path)
    gender_count[gender] += 1
    race_count[race] += 1

  gender_count = {gender: 1 / count if count > 0 else 0 for gender, count in gender_count.items()}
  race_count = {race: 1 / count if count > 0 else 0 for race, count in race_count.items()}

  for path in X_train:
    age, gender, race = get_labels(path)

    gender_prob = gender_count[gender] / len(X_train) * prob_tracker["gender"][gender]
    race_prob = race_count[race] / len(X_train) * prob_tracker["race"][race]

    prob = gender_prob * race_prob

    sample_probs.append(prob)

  sum_prob = sum(sample_probs)
  normalized_probs = [prob / sum_prob for prob in sample_probs]

  return normalized_probs

In [ ]:
def get_forget_retain_set(X_train, y_train, size, normalized_probs, is_forget=True, seed=42):
  """
  Returns the forgetset from the train_dataset

  Args:
    - X_train: X_train dataset (list of paths)
    - y_train: y_train dataset (list of ints representing bins)
    - size: percentage indicating what portion of the train_data dataset should be forget set
    - male_size: percentage indicating what portion of the forget_set should be male
  """

  np.random.seed(seed)
  num_samples = int(len(X_train) * size)
  forget_indices = np.random.choice(len(X_train), num_samples, p=normalized_probs, replace=False)

  if is_forget:
      X_forget = X_train[forget_indices]
      y_forget = y_train[forget_indices]
      return X_forget, y_forget
  else:
      retain_indices = np.setdiff1d(range(len(X_train)), forget_indices)

      X_retain = X_train[retain_indices]
      y_retain = y_train[retain_indices]

      return X_retain, y_retain

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_test_split(DATASET)


def load_example(image, age):
    result = {
        'image': image,
        # 'image_id': df_row['image_id'],
        'age_group': age // NUM_BINS,
        'age': age,
        # 'person_id': df_row['person_id']
    }
    return result


class HiddenDataset(Dataset):
    '''The hidden dataset.'''
    def __init__(self, probs, split='train', forget_size=0.01):
        super().__init__()
        self.examples = []

        self.data = [X_train, y_train]

        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

        if split != 'validation':
          self.data = get_forget_retain_set(self.data[0], self.data[1],
                                            normalized_probs=probs,
                                            size=forget_size,
                                            is_forget=split=='forget')

        for i in tqdm(range(len(self.data[0])), desc=f"Loading {split} data", leave=False):
          image_path = X_train[i]
          age = y_train[i]

          image = self.load_image(image_path)
          self.examples.append(load_example(image, age))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        image = example['image']
        image = image.to(torch.float32)
        example['image'] = image
        return example

    def load_image(self, image_path):
        image_path = str(image_path)
        image = cv.imread(image_path)
        if image is None:
          print(image_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB).transpose(2, 0, 1)
        image = torch.tensor(image, dtype=torch.float32)
        image = self.normalize(image)
        return image


def get_dataset(size=0.01, male_prob=0.5, race_probs=[0.2, 0.2, 0.2, 0.2, 0.2]):
    '''Get the dataset.'''
    normalized_probs = get_probs(X_train,
                                male_prob=male_prob,
                                race_probs=race_probs)

    retain_ds = HiddenDataset(normalized_probs, split='retain', forget_size=size)
    forget_ds = HiddenDataset(normalized_probs, split='forget', forget_size=size)
    val_ds = HiddenDataset(normalized_probs, split='validation')

    assert len(forget_ds) == int(len(X_train) * size)
    assert len(retain_ds) == len(X_train) - len(forget_ds)

    return retain_ds, forget_ds, val_ds

def get_dataloader(batch_size, retain_ds, forget_ds, val_ds):
  '''Get the dataloaders'''
  retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True)
  forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True)
  validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

  return retain_loader, forget_loader, validation_loader

## Original Model

In [ ]:
def create_model():
  model = resnet18(weights=None)
  model.fc = nn.Sequential(
      nn.Linear(model.fc.in_features, NUM_BINS),
      nn.LogSoftmax(dim=1)
      )
  model.to(DEVICE)
  return model

In [ ]:
def initialize_original_model(model, model_weights_path):
    checkpoint_path = os.path.join(ROOT_FOLDER, model_weights_path)
    checkpoint = torch.load(checkpoint_path)

    model.load_state_dict(checkpoint["model_state_dict"])

    return model

## Evaluation

In [ ]:
def accuracy(model, dataloader, progress=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # Set the model to evaluation mode

    num_correct = 0
    N = len(data)

    the_range = dataloader if progress is False else tqdm(dataloader, position=1, leave=False, desc="Evaluating")

    for example in the_range:
        images = example["image"]
        labels = example["age_group"]
        images = images.type(torch.float32).to(device).unsqueeze(0)
        labels = torch.tensor(labels).to(device)

        outputs = model(images)
        predicted_labels = outputs.argmax(dim=1)  # Assuming labels are class indices
        num_correct += (predicted_labels == labels).sum().item()

    return num_correct / N

In the following functions, retrained model refers to the model trained only on the retain set (and NOT the forget set)of the training set unlearned model refers to the model trained on the entire training set (retain + forget set), and then an unlearning algorithm was applied to model to "forget" the forget set.

In [ ]:
def effectiveness_test(retrained_model, unlearned_model, test_data):
    """Returns a scalar on how effective the unlearned model is at predicting the age of the test data, compared to the retrained model.

    The higher the scalar, the better the score.
    If the scalar is around 1, that means the unlearned model is at par with the retrained model.
    If the scalar is lower, then it means the retrained model is better at generalizing to the test data.
    If the scalar is higher, then it means the unlearned model is better at generalizing to the test data.
    """
    retrained_accuracy = accuracy(retrained_model, test_data)
    unlearned_accuracy = accuracy(unlearned_model, test_data)

    return unlearned_accuracy / retrained_accuracy if retrained_accuracy > 0 else 0

In [ ]:
def effectiveness_retain(retrained_model, unlearned_model, retain_data):
    """Returns a scalar on how effective the unlearned model is at predicting the age of the retain data, compared to the retrained model.

    The higher the scalar, the better the score.
    If the scalar is around 1, that means the unlearned model is at par with the retrained model.
    If the scalar is lower, then it means the retrained model is better at generalizing to the test data.
    If the scalar is higher, then it means the unlearned model is better at generalizing to the test data.
    """
    retrained_accuracy = accuracy(retrained_model, retain_data)
    unlearned_accuracy = accuracy(unlearned_model, retain_data)

    return unlearned_accuracy / retrained_accuracy  if retrained_accuracy > 0 else 0

In [ ]:
def certifiability(retrained_model, unlearned_model, forget_data):
    """Returns a scalar on how well the unlearned model has forgotten the forget data, as compared to the retrained model.

    The higher the disparity between the two forget accuracies, the higher the scalar and better the score.
    Note this metric does not go beyond 1.
    """
    retrained_accuracy = accuracy(retrained_model, forget_data)
    unlearned_accuracy = accuracy(unlearned_model, forget_data)

    return min(retrained_accuracy, unlearned_accuracy) / max(retrained_accuracy, unlearned_accuracy)  if max(retrained_accuracy, unlearned_accuracy) > 0 else 0


In [ ]:
def combined_score(score_test, score_retain, score_forget, weight_test=1, weight_retain=1, weight_forget=2):
    """Returns a combined score using the test, retain and forget scores.
    Each score is first raised to its weight, and then each of the resulting scores are multiplied together.

    The higher the score, the better a model performs.
    """
    return (score_test**weight_test) * (score_retain**weight_retain) * (score_forget**weight_forget)

# Unlearning Algorithms

In [ ]:
class UnlearningAlgorithm:
    def unlearning(self, net, retain_loader, forget_loader, val_loader):
        raise NotImplementedError

## Fanchuan

In [ ]:
# Utils
def kl_loss_sym(x,y):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    return kl_loss(nn.LogSoftmax(dim=-1)(x),y)

In [ ]:
class Fanchuan(UnlearningAlgorithm):
    def unlearning(self, net, retain_loader, forget_loader, val_loader):
        """Simple unlearning by finetuning."""
        epochs = 8
        retain_bs = 256
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.005,
                              momentum=0.9, weight_decay=0)

        optimizer_retain = optim.SGD(net.parameters(), lr=0.001*retain_bs/64, momentum=0.9, weight_decay=1e-2)
        ##the learning rate is associated with the batchsize we used
        optimizer_forget = optim.SGD(net.parameters(), lr=3e-4, momentum=0.9, weight_decay=0)

        total_step = int(len(forget_loader)*epochs)

        retain_ld = DataLoader(retain_loader.dataset, batch_size=retain_bs, shuffle=True)
        retain_ld4fgt = DataLoader(retain_loader.dataset, batch_size=256, shuffle=True)

        scheduler = CosineAnnealingLR(optimizer_forget, T_max=total_step, eta_min=1e-6)

        net.train()
        for sample in tqdm(forget_loader, desc="First Stage", leave=False): ##First Stage
            inputs = sample["image"]
            inputs = inputs.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            uniform_label = torch.ones_like(outputs).to(DEVICE) / outputs.shape[1] ##uniform pseudo label
            loss = kl_loss_sym(outputs, uniform_label) ##optimize the distance between logits and pseudo labels
            loss.backward()
            optimizer.step()

        net.train()
        for ep in tqdm(range(epochs), desc="Second Stage", leave=False): ##Second Stage
            net.train()
            for sample_forget, sample_retain in tqdm(zip(forget_loader, retain_ld4fgt), leave=False, total=len(forget_loader)):##Forget Round
                t = 1.15 ##temperature coefficient
                inputs_forget,inputs_retain = sample_forget["image"],sample_retain['image']
                inputs_forget, inputs_retain = inputs_forget.to(DEVICE), inputs_retain.to(DEVICE)
                optimizer_forget.zero_grad()
                outputs_forget,outputs_retain = net(inputs_forget),net(inputs_retain).detach()
                loss = (-1 * nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean() ##Contrastive Learning loss
                loss.backward()
                optimizer_forget.step()
                scheduler.step()

            for sample in tqdm(retain_ld, leave=False): ##Retain Round
                inputs, labels = sample["image"],sample["age_group"]
                labels = labels.type(torch.LongTensor)
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer_retain.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer_retain.step()

## Doun  Lee

In [ ]:
# Utils
class Masker(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, mask):
        ctx.save_for_backward(mask)
        return x

    @staticmethod
    def backward(ctx, grad):
        mask, = ctx.saved_tensors
        return grad * mask, None


class MaskConv2d(nn.Conv2d):
    def __init__(self, mask, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device='cpu'):
        super(MaskConv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                                     padding, dilation, groups, bias, padding_mode, device=device)
        self.mask = mask

    def forward(self, input):
        masked_weight = Masker.apply(self.weight, self.mask)
        return super(MaskConv2d, self)._conv_forward(input, masked_weight, self.bias)


def set_layer(model, layer_name, layer):
    splited = layer_name.split('.')
    if len(splited) == 1:
        setattr(model, splited[0], layer)
    elif len(splited) == 3:
        setattr(getattr(model, splited[0])[int(splited[1])], splited[2], layer)
    elif len(splited) == 4:
        getattr(getattr(model, splited[0])[int(splited[1])], splited[2])[int(splited[3])] = layer


def get_grads_for_snip(model, retain_loader, forget_loader):
    indices = torch.randperm(len(retain_loader.dataset), dtype=torch.int32, device='cpu')[:len(forget_loader.dataset)]
    retain_dataset = Subset(retain_loader.dataset, indices)
    retain_loader = DataLoader(retain_dataset, batch_size=64, shuffle=True)

    model.zero_grad()
    for sample in tqdm(retain_loader, leave=False):
        inputs = sample["image"]
        targets = sample["age_group"]
        targets = targets.long()
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()

    for sample in tqdm(forget_loader, leave=False):
        inputs = sample["image"]
        targets = sample["age_group"]
        targets = targets.long()
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        outputs = model(inputs)
        loss = -F.cross_entropy(outputs, targets)
        loss.backward()


@torch.no_grad()
def replace_maskconv(model):
    for name, m in tqdm(list(model.named_modules()), leave=False):
        if isinstance(m, MaskConv2d):
            conv = nn.Conv2d(m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            conv.weight.data = m.weight
            conv.bias = m.bias
            set_layer(model, name, conv)


@torch.no_grad()
def re_init_model_snip_ver2_little_grad(model, px): # re init smallest gradients
    for name, m in list(model.named_modules()):
        if isinstance(m, nn.Conv2d):
            mask = torch.zeros_like(m.weight, device=DEVICE).bool()
            nparams_toprune = round(px*mask.nelement())

            out_c, in_c, ke, _ = mask.shape
            value = -m.weight.grad.abs()
            topk = torch.topk(value.view(-1), k=nparams_toprune)
            mask.view(-1)[topk.indices] = True
            grad_mask = mask.clone().float()
            grad_mask[grad_mask==0] += 0.1

            new_conv = MaskConv2d(grad_mask, m.in_channels, m.out_channels, m.kernel_size, m.stride,
                 m.padding, m.dilation, m.groups, m.bias!=None, m.padding_mode, device=DEVICE)
            nn.init.kaiming_normal_(new_conv.weight, mode="fan_out", nonlinearity="relu")

            new_conv.weight.data[~mask] = m.weight[~mask]

            set_layer(model, name, new_conv)


class LinearAnnealingLR(_LRScheduler):
    def __init__(self, optimizer, num_annealing_steps, num_total_steps):
        self.num_annealing_steps = num_annealing_steps
        self.num_total_steps = num_total_steps

        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_annealing_steps:
            return [base_lr * self._step_count / self.num_annealing_steps for base_lr in self.base_lrs]
        else:
            return [base_lr * (self.num_total_steps - self._step_count) / (self.num_total_steps - self.num_annealing_steps) for base_lr in self.base_lrs]

In [ ]:
class DounLee(UnlearningAlgorithm):
  def unlearning(self,
      net,
      retain_loader,
      forget_loader,
      val_loader):
      replace_maskconv(net)
      get_grads_for_snip(net, retain_loader, forget_loader)
      re_init_model_snip_ver2_little_grad(net, 0.3)

      epochs = 5
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.SGD(net.parameters(), lr=0.001,
                        momentum=0.9, weight_decay=5e-4)
      scheduler = LinearAnnealingLR(optimizer, num_annealing_steps=(epochs+1)//2, num_total_steps=epochs+1)

      net.train()

      for ep in tqdm(range(epochs), desc="Unlearing Epochs", leave=False):
          for sample in tqdm(retain_loader, leave=False):
              inputs = sample["image"]
              targets = sample["age_group"]
              targets = targets.long()
              inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

              optimizer.zero_grad()
              outputs = net(inputs)
              loss = criterion(outputs, targets)

              loss.backward()
              optimizer.step()

          scheduler.step()

      net.eval()

## Seif Eddine Achour

In [ ]:
# Utils
class CustomCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights=None):
        super(CustomCrossEntropyLoss, self).__init__()
        self.class_weights = class_weights

    def forward(self, input, target):
        # Compute the standard cross-entropy loss
        ce_loss = nn.functional.cross_entropy(input, target)

        # Apply class weights to the loss if provided
        if self.class_weights is not None:
            # Calculate the weights for each element in the batch based on the target
            weights = torch.tensor([self.class_weights[i] for i in target], device=input.device)
            ce_loss = torch.mean(ce_loss * weights)

        return ce_loss


def vision_confuser(model, std = 0.6):
    for name, module in model.named_children():
        if hasattr(module, 'weight'):
            if 'conv' in name:

                actual_value = module.weight.clone().detach()
                new_values = torch.normal(mean=actual_value, std=std)
                module.weight.data.copy_(new_values)

In [ ]:
class SeifEddine(UnlearningAlgorithm):
  def unlearning(self, net, retain_loader, forget_loader, validation_loader):
    vision_confuser(net)

    epochs = 4
    w = 0.05
    class_weights = [1, w, w, w, w, w, w, w, w, w]

    criterion = CustomCrossEntropyLoss(class_weights)
    optimizer = optim.SGD(net.parameters(), lr=0.0007,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

    net.train()
    i=0

    for ep in tqdm(range(epochs), desc="Unlearning Epochs", leave=False):
        i=0
        net.train()
        for sample in retain_loader:
            inputs = sample["image"]
            targets = sample["age_group"]
            targets = torch.tensor(targets)
            targets = targets.long()
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        if (ep == epochs-2):
            vision_confuser(net , 0.005) # increase model robustness before last training epoch

        scheduler.step()

    net.eval()

## Run Block

In [ ]:
# algorithm = Fanchuan()
# n_checkpoints = 1

In [ ]:
# retain_loader, forget_loader, validation_loader = get_dataset(BATCH_SIZE)

In [ ]:
# os.makedirs('/tmp', exist_ok=True)

# net = create_model()

# for i in range(n_checkpoints):
#     net = initialize_original_model(net, "checkpoints/checkpoint_29")
#     algorithm.unlearning(net, retain_loader, forget_loader, validation_loader)
#     state = net.state_dict()
#     torch.save(state, f'/tmp/unlearned_checkpoint_{i}.pth')

# subprocess.run('zip submission.zip /tmp/*.pth', shell=True)

# Experiments

## Experiment Variables


In [ ]:
LOADING_CHECKPOINT = 29

SIZES = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5]
MALE_PROBS = [0.3, 0.5, 0.7]
RACE_PROBS = [[0.2, 0.2, 0.2, 0.2, 0.2], [0.6, 0.1, 0.1, 0.1, 0.1],
              [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1],
              [0.1, 0.1, 0.1, 0.6, 0.1], [0.1, 0.1, 0.1, 0.1, 0.6]]

default_size = 0.02
default_gender = 0.5
default_race = [0.2, 0.2, 0.2, 0.2, 0.2]

SIZE_VARIANT = [
    {
        "name": f"{int(size * 100)}-size_{int(default_gender * 100)}-male_{'-'.join([str(int(prob * 100)) for prob in default_race])}-race",
        "size": size,
        "male_prob": default_gender,
        "race_probs": default_race,
    }
    for size in SIZES
]

GENDER_VARIANT = [
    {
        "name": f"{int(default_size * 100)}-size_{int(male_prob * 100)}-male_{'-'.join([str(int(prob * 100)) for prob in default_race])}-race",
        "size": default_size,
        "male_prob": male_prob,
        "race_probs": default_race,
    }
    for male_prob in MALE_PROBS
]

RACE_VARIANT = [
    {
        "name": f"{int(default_size * 100)}-size_{int(default_gender * 100)}-male_{'-'.join([str(int(prob * 100)) for prob in race])}-race",
        "size": default_size,
        "male_prob": default_gender,
        "race_probs": race,
    }
    for race in RACE_PROBS
]

FORGET_VARIANTS = SIZE_VARIANT + GENDER_VARIANT + RACE_VARIANT


TEST_VARIANT = [
    {
        "name": f"{int(0.1 * 100)}-size_{int(default_gender * 100)}-male_{'-'.join([str(int(prob * 100)) for prob in default_race])}-race",
        "size": 0.1,
        "male_prob": default_gender,
        "race_probs": default_race,
    }
]

ALGORITHMS = [Fanchuan(), DounLee(), SeifEddine()]

In [ ]:
import json

def run_experiments(forget_variants, varname):
  """Runs all experiments using the forget_variants"""

  os.makedirs(os.path.join(ROOT_FOLDER, 'experiments'), exist_ok=True)

  experiment_results = []

  for variant in tqdm(forget_variants, desc="Variant Progress"):
    checkpoint_path = lambda x : os.path.join(ROOT_FOLDER, f'experiments/{type(x).__name__}_{variant["name"]}_results.json')
    if all(os.path.exists(checkpoint_path(algo)) for algo in ALGORITHMS):
       continue

    name, size, male_prob, race_probs = variant["name"], variant['size'], variant['male_prob'], variant['race_probs']
    retain_ds, forget_ds, validation_ds = get_dataset(size=size, male_prob=male_prob, race_probs=race_probs)
    retain_loader, forget_loader, validation_loader = get_dataloader(BATCH_SIZE, retain_ds, forget_ds, validation_ds)

    for algorithm in tqdm(ALGORITHMS, desc="Algorithm Progress", leave=False):
      if os.path.exists(checkpoint_path(algorithm)):
        continue

      # Do unlearning
      net = create_model()
      net = initialize_original_model(net, f"checkpoints/checkpoint_{LOADING_CHECKPOINT}")

      algorithm.unlearning(net, retain_loader, forget_loader, validation_loader)

      state = net.state_dict()
      torch.save(state, os.path.join(ROOT_FOLDER, f'experiments/{type(algorithm).__name__}_{variant["name"]}.pth'))

      # Evaluate
      retrained_model = create_model()
      retrained_model = initialize_original_model(retrained_model, f"{variant['name']}/checkpoint_{LOADING_CHECKPOINT}")

      test_effectiveness_score = effectiveness_test(retrained_model, net, validation_ds)
      retain_effectiveness_score = effectiveness_retain(retrained_model, net, retain_ds)
      certifiability_score = certifiability(retrained_model, net, forget_ds)
      combined_test_score = combined_score(test_effectiveness_score, retain_effectiveness_score, certifiability_score)

      results = {
          "name": name,
          "algorithm": type(algorithm).__name__,
          "test_effectiveness_score": test_effectiveness_score,
          "retain_effectiveness_score": retain_effectiveness_score,
          "certifiability_score": certifiability_score,
          "combined_test_score": combined_test_score
      }

      results_file_path = os.path.join(ROOT_FOLDER, f'experiments/{type(algorithm).__name__}_{variant["name"]}_results.json')

      with open(results_file_path, 'w') as f:
        json.dump(results, f)

      experiment_results.append(results)

  os.makedirs(os.path.join(ROOT_FOLDER, 'experiments/total'), exist_ok=True)
  final_results_path = os.path.join(ROOT_FOLDER, f'experiments/total/{varname}_results.json')

  with open(final_results_path, 'w') as f:
      json.dump(experiment_results, f)

  return experiment_results

## Run experiments using algorithms


In [ ]:
run_experiments(GENDER_VARIANT, "gender")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
run_experiments(RACE_VARIANT, "race")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
run_experiments(SIZE_VARIANT, "size")

Variant Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Loading retain data:   0%|          | 0/9483 [00:00<?, ?it/s]

Loading forget data:   0%|          | 0/9482 [00:00<?, ?it/s]

Loading validation data:   0%|          | 0/18965 [00:00<?, ?it/s]

Algorithm Progress:   0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/9483 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/9483 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/9482 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/9482 [00:00<?, ?it/s]

[{'name': '50-size_50-male_20-20-20-20-20-race',
  'algorithm': 'Fanchuan',
  'test_effectiveness_score': 1.578706401398485,
  'retain_effectiveness_score': 1.5768207515796477,
  'certifiability_score': 0.6341489137312802,
  'combined_test_score': 1.0010740472679875},
 {'name': '50-size_50-male_20-20-20-20-20-race',
  'algorithm': 'DounLee',
  'test_effectiveness_score': 1.5784566719387332,
  'retain_effectiveness_score': 1.5768207515796477,
  'certifiability_score': 0.6341489137312802,
  'combined_test_score': 1.0009156912362553},
 {'name': '50-size_50-male_20-20-20-20-20-race',
  'algorithm': 'SeifEddine',
  'test_effectiveness_score': 1.5782069424789813,
  'retain_effectiveness_score': 1.576321915530429,
  'certifiability_score': 0.6343496149382846,
  'combined_test_score': 1.0010740973841814}]